In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture as GMM
from matplotlib import pyplot as plt, cm as cm, mlab as mlab
import seaborn as sns; sns.set()
import progressbar as pb
import time 

In [2]:
# read csv/excel data files 
pnas_data1 = pd.read_csv('/home/jaeweon/research/data/pnas_data1.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# format data 

# extract 9 Complexity Characteristic variables 
features = ['PolPop', 'PolTerr', 'CapPop', 'levels', 'government','infrastr', 'writing', 'texts', 'money']

# take subset of original data table with 9 CCs and change it into numpy array 
data_mat = pnas_data1.loc[:, features].values
scaled = StandardScaler().fit_transform(data_mat)
P, D, Q = svd(scaled)
data = np.matmul(scaled, Q.T) # data matrix is obtained by multiplying initial data matrix with SVD column matrix

NameError: name 'svd' is not defined

In [2]:
# class for progressbar
class progress_timer:

    def __init__(self, n_iter, description="Something"):
        self.n_iter = n_iter
        self.iter = 0
        self.description = description + ': '
        self.timer = None
        self.initialize()

    def initialize(self):
        #initialize timer
        widgets = [self.description, pb.Percentage(), ' ',   
                   pb.Bar(marker=pb.RotatingMarker()), ' ', pb.ETA()]
        self.timer = pb.ProgressBar(widgets=widgets, maxval=self.n_iter).start()

    def update(self, q=1):
        #update timer
        self.timer.update(self.iter)
        self.iter += q

    def finish(self):
        #end timer
        self.timer.finish()

In [4]:
# angle between vectors and various measurements for errors 

def angle(vec1, vec2):
    """
    Given two vectors, compute the angle between the vectors
    """
    assert vec1.shape == vec2.shape
    
    cos_vec = np.inner(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
    angle = math.acos(cos_vec)
    in_deg = math.degrees(angle)
    if in_deg >= 90:
        return (180-in_deg)
    return in_deg

def angle_error(ang1, ang2):
    """
    Given two angles, compute the relative error
    """
    if ang1 >= 90 and ang2 < 90:
        return abs((180-ang1)-ang2)/(180-ang1)
    elif ang1 < 90 and ang2 >= 90:
        return abs(ang1-(180-ang2))/(ang1)
    elif ang1 >= 90 and ang2 >= 90:
        return abs((180-ang1)-(180-ang2))/(180-ang1)
    else:
        return abs(ang1-ang2)/ang1

def vec_error(original, bst):
    """
    Given two n-dim vectors, one from the original and one from bootstrapping, compute the relative error
    between these two vectors
    """
    assert original.shape == bst.shape 
    
    return np.linalg.norm(original-bst)/np.linalg.norm(original)

def error(val1, val2):
    """
    Given two values, compute the relative error
    """
    return abs(val1-val2)/val1


In [ ]:
#eigenvalues and eigenvectors for covariance matrix. Determine which of the two components they belong to by 
#the weights of the gaussians

def eig(mat):
    """
    Given a list of two matrices, compute the largest eigenvalues' proportion over sum of all eigenvalues as well 
    as the corresponding eigenvectors
    """
    eig_val1, eig_vec1 = np.linalg.eig(mat[0])
    eig_val2, eig_vec2 = np.linalg.eig(mat[1])
    
    assert eig_vec1.shape == eig_vec2.shape
    assert len(eig_val1) == len(eig_val2)
    
    # proportion of largest eigenvalue with respect to each component
    val1 = max(eig_val1)/sum(eig_val1)
    val2 = max(eig_val2)/sum(eig_val2)
    
    # eigenvector corresponding to the largest eigenvalue with respect to each component
    vec1 = eig_vec1[:,np.argmax(eig_val1)]
    vec2 = eig_vec2[:,np.argmax(eig_val2)]
    
    assert vec1.shape == vec2.shape
    
    return val1, val2, vec1, vec2


def weight_set(cov_mat):
    """
    Given gmm model, set the component with larger weight to be the first component and the component with
    smaller weight to be the second component
    """
    
    val1, val2, vec1, vec2 = eig(cov_mat) #eigenvalues and eigenvecotrs for covariance matrices

    # designate the larger component to be the component1 and the smaller component the component2 
    if gmm.weights_[0] > gmm.weights_[1]:
        return val1, val2, vec1, vec2
    else:
        return val2, val1, vec2, vec1


In [ ]:
# Fit bivariate Gaussian on the original data and compute confidence intervals, error bars using bootstrapping
pt = progress_timer(description= 'boostrapping', n_iter=5000)

#Gaussian Mixture Model 
#fit GMM
gmm = GMM(n_components=2).fit(data_mat)
cov = gmm.covariances_

eigval1, eigval2, eigvec1, eigvec2 = weight_set(cov)
orig_angle = angle(eigvec1, eigvec2) # angle between the main eigenvectors 

# boostrapping for mean of gmm
def bstr(data, n=5000):
    """
    Given data matrix, perform bootstrapping by collecting n samples (default = 5000) and return the 
    error rate for the mean of the data. Assume that the given data matrix is numpy array
    """    
    perc_one = list() # hold proprtion of largest eigenvalues for component 1
    perc_two = list() # hold propotion of largest eigenvalues for component 2
    
    vec_one = list() # hold main eigenvectors for component 1
    vec_two = list() # hold main eigenvectors for component 2
    
    angles = list() # hold angle between main eigenvectors of two Gaussian component
    
    for i in range(n):
        # resample from the original data matrix
        resampled_mat = data[np.random.randint(0, data.shape[0])]
        
        for j in range(data.shape[0]-1):
            data_resample = data[np.random.randint(0, data.shape[0])]
            resampled_mat = np.vstack((resampled_mat, data_resample))
    
        assert resampled_mat.shape == data.shape # check if the resampled matrix has same dimension as data matrix
        
        gmm = GMM(n_components=2).fit(resampled_mat)
        gmm_cov = gmm.covariances_
        
        eig_val1, eig_val2, eig_vec1, eig_vec2 = weight_set(gmm_cov)
        
        perc_one.append(eig_val1)
        perc_two.append(eig_val2)
        vec_one.append(eig_vec1)
        vec_two.append(eig_vec2)
        angles.append(angle(eig_vec1, eig_vec2))
        
        pt.update()
    
#     perc_one = sum(perc_one)/len(perc_one)
#     perc_two = sum(perc_two)/len(perc_two)
    
    vec_one = np.mean(vec_one, axis=0)
    vec_two = np.mean(vec_two, axis=0)
    
#     bstr_angle = sum(angles)/len(angles)
    
    return perc_one, perc_two, vec_one, vec_two, angles

perc_one, perc_two, vec_one, vec_two, bstr_angle = bstr(data_mat)
pt.finish()

# plot the histogram for eigenvalues and angles
num_bins = 200
itera = [perc_one, perc_two, bstr_angle]
for i in range(len(itera)):
    n, bins, patches = plt.hist(itera[i], num_bins, normed=1, facecolor='blue', alpha=0.5)
    if i == 0:
        plt.title('histogram for bootstrapped eigenvalues of component 1')
        plt.axvline(x=eigval1)
    elif i == 1:
        plt.title('histogram for bootstrapped eigenvalues of component 2')
        plt.axvline(x=eigval2)
    else:
        plt.title('histogram for bootstrapped angles')
        plt.axvline(x=orig_angle)
    
    plt.legend()
    plt.show()
    plt.close()
